In [2]:
print('common 1.5.1')

common 1.5.1


In [3]:
import numpy as np
import matplotlib.pyplot as plt


def plotStats(xlen, trainEpochs, trainAccs, trainLosses, earlyStopACCmin=1.0):
    print(min(trainEpochs))
    print(max(trainEpochs))
    print(np.mean(trainEpochs))
    print(np.std(trainEpochs))

    fig, ax1 = plt.subplots()
    plt.title('Epochs to reach acc={}% or best'.format(earlyStopACCmin*100))
    plt.xlabel('run')
    ax1.bar(range(xlen), trainEpochs)
    ax1.set_ylabel('epochs')

    ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
    ax2.plot(range(xlen), [x * 100 for x in trainAccs], 'red')
    ax2.set_ylabel('Acc(%)', color='red')
    ax2.tick_params(axis='y', labelcolor='red') #set axis tick mark labels to red
    plt.show()
    
    plt.hist(trainEpochs, bins=60)
    plt.title('Histogram of epochs/runs to reach acc={}% or best'.format(earlyStopACCmin*100))
    plt.ylabel('runs')
    plt.xlabel('epochs')
    plt.show()

    '''
    plt.plot(range(xlen), trainAccs)
    plt.title('Validation Acc for each run')
    plt.ylabel('Acc(%)')
    plt.xlabel('run')
    plt.show()

    plt.plot(range(xlen), trainLosses)
    plt.title('Validation loss for each run')
    plt.ylabel('loss')
    plt.xlabel('run')
    plt.show()
    '''

#trainEpochs = [109, 26, 65, 28, 23, 167, 34, 54, 30, 50, 57, 200, 200, 200, 200, 41, 42, 67, 66, 200, 21, 200, 41, 179, 33]
#trainAccs   = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9992857, 0.9992857, 0.9992857, 0.9992857, 1.0, 1.0, 1.0, 1.0, 0.9992857, 1.0, 0.9992857, 1.0, 1.0, 1.0]
#trainLosses = [0.00015383255714009785, 0.00013310836369031902, 0.00019599902986133983, 9.929392099870268e-05, 0.00021146672146035226, 9.084118245922098e-05, 0.00012753754036657773, 0.0001668362780031614, 0.00012830220534581035, 0.0003216641377430479, 0.00022237549194116594, 0.00013495464647501367, 9.006860194003049e-05, 0.0001930042541080397, 7.954422169238537e-05, 0.00021182476362219165, 5.630278161991108e-05, 0.00019072362820990032, 0.00011617205233686352, 0.00010403195843203467, 0.00017226923697384177, 0.0001461460787775388, 0.00012822973083313999, 0.00011064412045147774, 0.00010872509995949084]

#plotStats(25, trainEpochs, trainAccs, trainLosses, earlyStopACCmin=1.0)

In [4]:
def plotBoxes1(logseries, title=''):
    boxes = []
    for e in logseries: 
      if e != 200:
        boxes.append(logseries[e]['epoch'])

    #plt.subplot(1,2,1)
    plt.boxplot(boxes, positions=[20, 18, 16, 14, 12, 10, 8, 6, 4], widths=1)
    plt.xlabel('points by channel x 100')
    plt.ylabel('epochs')
    plt.title(title)
    #plt.subplot(1,2,2)
    plt.show()
    plt.boxplot(logseries[200]['epoch'], positions=[200])
    plt.ylabel('epochs')
    plt.xlabel('points by channel')
    plt.title(title)
    plt.show()

In [5]:
def plotBoxes2(logseries, measure='acc', title='', width=1, positions=None):
    boxes = []
    for e in logseries:
        boxes.append(logseries[e][measure])

    plt.boxplot(boxes, positions=positions, widths=width, showmeans=True)
    plt.xlabel('points by channel x 100')
    plt.ylabel('val_acc')
    plt.title(title)
    plt.show()

In [6]:
import time, pytz, pickle
from datetime import datetime

class SimpleLog:   
    def __init__(self, tz='Europe/Lisbon'):
        self.clear()
        self.__tz = pytz.timezone(tz)
    
    def copy(self):
        copyLog = SimpleLog()
        copyLog._SimpleLog__tz = self.__tz
        copyLog._SimpleLog__log = self.__log.copy()
        return copyLog

    def len(self):
        return len(self.__log)
        
    def add(self, data):
        now = datetime.now(self.__tz)
        self.__log.append((now, data))
    
    def getAll(self):
        return self.__log
      
    def getFirst(self):
        if len(_self.__log <= 0): return None
        return self.__log[0]
      
    def getLast(self):
        if len(self.__log) <= 0: return None
        return self.__log[len(self.__log)-1]
    
    def getTime(self):
        return [e[0] for e in self.__log]
    
    def getInfo(self):
        return [e[1] for e in self.__log]
    
    def clear(self):
        self.__log = []
    
    def save(self, fn):
        with open(fn, 'wb') as fp:
            pickle.dump(self, fp)
    
    @classmethod 
    def load(cls, fn):
        with open(fn, 'rb') as fp:
            return pickle.load(fp)

    #Appends to receptor
    def append(self, log):
        self.__log.extend(log._SimpleLog__log)
        
    #Returns new Log with head(n) entries of receptor
    def head(self, howMany=1):
        nLog = self.copy()  #Could just create new an copy __tz
        nLog._SimpleLog__log = self.__log[:howMany]
        return nLog

    #Returns new Log with tail(n) entries of receptor
    def tail(self, howMany=1):
        nLog = self.copy()  #Could just create new an copy __tz
        nLog._SimpleLog__log = self.__log[-howMany:]
        return nLog

In [7]:
#Get logs experience series
#(1200, 16, 4, 0.9771429, 0.005391452915833464, time)

def getLogSeries(logFNL, getTime=False):
    series = {}

    log = SimpleLog.load(logFNL)

    #for e in [(2,0,1),(2,0,2),(2,0,4),(6,0,2)]:
    for e in log.getInfo():
        key   = e[0]   #samples, or points, or ...
        epoch = e[2]
        acc   = e[3]
        loss  = e[4]
        if getTime: runtime  = e[5] 
        if key in series:
            series[key]['epoch'].append(epoch)
            series[key]['acc'].append(acc)
            series[key]['loss'].append(loss)
            if getTime: series[key]['runtime'].append(runtime)
        else:
            if getTime: series[key] = {'epoch':[epoch], 'acc':[acc], 'loss':[loss], 'runtime':[runtime]}
            else:       series[key] = {'epoch':[epoch], 'acc':[acc], 'loss':[loss]}
    return series

#logFNL = recordpath + 'experiment.log'
#logseries = getLogSeries(logFNL, True) 
'''
for e in logseries: 
    print(e,':')
    print(logseries[e]['epoch']) 
    print(logseries[e]['acc']) 
    print(logseries[e]['loss']) 
    print(logseries[e]['runtime']) 
'''

"\nfor e in logseries: \n    print(e,':')\n    print(logseries[e]['epoch']) \n    print(logseries[e]['acc']) \n    print(logseries[e]['loss']) \n    print(logseries[e]['runtime']) \n"

In [8]:
def lstats(lseries, measure='acc', best=1.0):
    exps  = []
    runs  = []
    means = []
    maxs  = []
    mins  = []
    stdevs= []
    nbests= []

    #extract list measure from logseries
    for exp in lseries:
        list    = lseries[exp][measure]

        #get stats for each experiment
        exps.append(exp)
        runs.append(len(list))
        means.append(np.mean(list))
        stdevs.append(np.std(list))
        maxs.append(max(list))
        mins.append(min(list))
        nbests.append(sum(x == best for x in list))

    return exps, runs, means, mins, maxs, stdevs, nbests

In [9]:
#if mode=None (default),                      Stalled val_acc epochs is NOT changed (is first stall epoch + epochPatience
#if mode=('max', maxEpochs, epochPatience),   Stalled val_acc epochs is set to maxEpochs
#if mode=('first', maxEpochs, epochPatience), Stalled val_acc epochs is set to epoch - epochPatience

#(1200, 16, 4, 0.9771429, 0.005391452915833464, time, memory)

def showMeans(lseries, maxAcc=1.0, mode=None, time=False):
    print('Exp   runs 100%    best     val_acc (stdev)    epochs (stdev)    time')
    
    for e in lseries: 
        epochs = lseries[e]['epoch'].copy()      #Do not change passed logseries epochs
        accs   = lseries[e]['acc']
        loss   = lseries[e]['loss']
        if time: runtime = lseries[e]['runtime']

        runs   = len(epochs)

        if mode is not None:
            maxEpochs = mode[1]
            epatience = mode[2]  
            for i in range(runs):
                if (accs[i] < maxAcc and epochs[i] < maxEpochs):
                    if mode[0] == 'max':    #Set stall acc epochs for maxEpochs
                        epochs[i] = maxEpochs
                    if mode[0] == 'first':  #Set stall acc epochs for first stalled epoch (subtract epatience)
                        epochs[i] = epochs[i] - epatience             

        mepoch  = np.mean(epochs)
        sepoch  = np.std(epochs)
        macc    = np.mean(accs)
        sacc    = np.std(accs)
        bestacc = max(accs)
        acc100  = sum(x == maxAcc for x in accs)
        mloss   = np.mean(loss)
        
        if time: mtime = np.mean(runtime)

        print('{:>4} {:4d} {:4d}   {:8.4f}  {:8.4f} ({:6.4f})  {:7.2f} ({:5.2f})'.format(e, runs, acc100, bestacc*100, macc*100, sacc*100, mepoch, sepoch), end='')
        if time: print('   {:5.2f}'.format(mtime))
        else:    print()
